学生証番号 :

名前 :

メールアドレス(法政大学) :

# B1-3　Back-propagation neural network (BPNN)を用いた画像認識問題

$
\newcommand{\bm}[1]{{\bf #1}}
\newcommand{\vt}[1]{{\boldsymbol #1}}
$

先ほどと同様に、BPNNを用いて今度は小さいカラー画像の認識を扱う。

ここで扱うCIFAR10（Canadian Institute For Advanced Research）データセットは32$\times$32のカラー画像を10種類に分類するタスクのためのデータセットである。


- ラベル「0」： airplane（飛行機）
- ラベル「1」： automobile（自動車）
- ラベル「2」： bird（鳥）
- ラベル「3」： cat（猫）
- ラベル「4」： deer（鹿）
- ラベル「5」： dog（犬）
- ラベル「6」： frog（カエル）
- ラベル「7」： horse（馬）
- ラベル「8」： ship（船）
- ラベル「9」： truck（トラック）


## 準備

In [ ]:
# [1-0]
# ライブラリのインポート
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

### CIFAR-10 dataset のダウンロードと形状の確認

In [ ]:
# [1-1]
# データを呼び出す関数の作成。
# 内容はB1-2のget_data_mnistと同じ
def get_data_cifar(flatten=True, data_size=None):
    (X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()
    # 0 ~ 1の範囲に正規化を行う
    X_train = X_train.astype(np.float32) / 255.0
    X_test = X_test.astype(np.float32) / 255.0
    y_train = y_train.flatten()
    y_test = y_test.flatten()
    if data_size is not None:
        X_train = X_train[:data_size]
        y_train = y_train[:data_size]
    if flatten:
        X_train = X_train.reshape(len(X_train), -1)
        X_test = X_test.reshape(len(X_test), -1)
    return (X_train, y_train), (X_test, y_test)

In [ ]:
# [1-2]
# B1-2のget_data_mnist
(X_train, y_train), (X_test, y_test) = get_data_cifar()

# ダウンロードしたデータの大きさ（次元数）を確認する。
print(type(X_train))
print("x_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

# 学習、テスト用データの数をそれぞれ、保存しておく。
num_train = X_train.shape[0]
num_test = X_test.shape[0]

In [ ]:
# [1-3]
# 予測ラベルの名前を所持しておく
target_names = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]
# B1-2のrandom_plotと同じ
def random_plot(X, y, predict=None):
    W = 10
    H = 5
    fig = plt.figure(figsize=(10, 20))
    fig.subplots_adjust(left=0, right=1, bottom=0, top=0.3, hspace=0.10, wspace=0.10)
    for i in range(5):
        for j in range(10):
            x_tmp = X[y == j]
            idx = np.random.randint(len(x_tmp))
            x = x_tmp[idx].reshape(32, 32, 3)
            ax = fig.add_subplot(H, W, (i * 10) + j + 1, xticks=[], yticks=[])
            ax.imshow(x)
            if predict is not None:
                pred_tmp = predict[y == j]
                p = pred_tmp[idx]
                ax.set_title(f"{j} -> {p}")
            else:
                ax.set_title(target_names[j])

In [ ]:
# [1-4]
# 学習用データを覗いてみる。
# 　複数回実行すると画像も変化する。
random_plot(X_train, y_train)

In [ ]:
# [1-5]
# テストデータを覗いてみる。
random_plot(X_test, y_test)

In [ ]:
# [1-6]
# BPNNに入力するために、各データを1次元化したデータを呼び出す。
(X_train, y_train), (X_test, y_test) = get_data_cifar(
    flatten=True
)  # defaultがTrueなので渡す必要はない
# 一次元にしたデータを確認
print(f"X_train_flat : {X_train.shape}")
print(f"X_test_flat  : {X_test.shape}")

## BPNNモデルの定義

10クラス分類問題なので、出力層のノード数は10。
中間層（隠れ層）のノード数は、前回同様ハイパーパラメータ（モデルを定義する、学習する前に決めておくべきパラメータ）である。  
(なぜ入力が3072かは大丈夫？）

(input) 3072 - 50 - 10 (output)

In [ ]:
# [1-7]
model = keras.Sequential(
    [
        layers.InputLayer(input_shape=(3072,)),
        layers.Dense(128, activation="relu"),
        layers.Dense(10, activation="softmax"),
    ]
)
model.summary()

In [ ]:
# [1-8]
# 誤差関数と最適化手法を設定する関数を作成する。
def model_setup(model, optimizer="adam", lr=0.001):
    # optimizerは再急降下法の改良アルゴリズムであり、SGDよりも性能が高い。　（細かい説明は省略）
    if optimizer == "adam":
        optim = keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == "sgd":
        optim = keras.optimizers.SGD(learning_rate=lr)
    else:
        raise ValueError
    # sparse_categorical_crossentropy　はcross entropyに対応している。(詳細は省く)
    # 性能評価の基準には"accuracy"（正解率）を用いる。
    model.compile(
        loss="sparse_categorical_crossentropy", optimizer=optim, metrics=["accuracy"]
    )

In [ ]:
# [1-9]
# モデルのセットアップ
model_setup(model)

# テストデータで評価する
loss_no_train, accuracy_no_train = model.evaluate(X_test, y_test, verbose=0)
print("学習前の誤差 : ", loss_no_train)
print("学習前の正解率 : ", accuracy_no_train)

## 学習




In [ ]:
# [1-10]
# 学習を実行する。
# パラメータの設定
batch_size = 128
epochs = 10
# 　学習回数は上で10回と定義している。
trainlog = model.fit(
    X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1
)

Draw training curves.

In [ ]:
# [1-11]
# 学習曲線を描画する関数の作成
def plot_result(log):
    fig = plt.figure(figsize=(10, 3))
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.plot(log.history["accuracy"])
    ax1.plot(log.history["val_accuracy"])
    ax1.set_title("Model accuracy")
    ax1.set_ylabel("Accuracy")
    ax1.set_xlabel("Epoch")
    ax1.legend(["Train", "Validation"], loc="best")
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.plot(log.history["loss"])
    ax2.plot(log.history["val_loss"])
    ax2.set_title("Model loss")
    ax2.set_ylabel("Loss")
    ax2.set_xlabel("Epoch")
    ax2.legend(["Train", "Validation"], loc="best")
    plt.show()

In [ ]:
# [1-12]
# 実際に学習曲線の確認
# 右が正解率を左が誤差の推移を表している。
plot_result(trainlog)

In [ ]:
# [1-13]
# 評価
loss_train, accuracy_train = model.evaluate(X_test, y_test, verbose=0)

# 学習前のスコアの表示
print("学習前の誤差 : ", loss_no_train)
print("学習前の正解率 : ", accuracy_no_train)

# 学習後のスコアの表示
print("===== 学習後 =====")
print(f"学習後の誤差 :  ", loss_train)
print(f"学習後の正解率 : ", accuracy_train)

先ほどと同様、学習に伴い性能は上がっているが、MNISTに比べると精度はよくない。

In [ ]:
# [1-14]
# それぞれのデータの予測を確認する。
# モデルの予測を取得。
test_predict = np.argmax(model.predict(X_test), 1)
# random_plot関数のpredictに予測を渡す。
random_plot(X_test, y_test, test_predict)
# 実際のラベル -> 予測されたラベル　になっている。

In [ ]:
# [1-15]
# 間違えたものに絞って表示
false_idx = y_test != test_predict
# 間違えたもののみを描画
random_plot(X_test[false_idx], y_test[false_idx], test_predict[false_idx])

MNISTの時とは異なり、認識が難しいものだけではなく、比較的に簡単に識別できそうなデータが多く存在する。

##  課題B1-3 性能向上のための試行錯誤

今回のモデルの性能をあげるにはどうしたらいいか？
自分でいろいろと考えて、動かしてみよ。

性能が上げられなくても、あるいはあがる量がわずかであっても、それについてしっかり考察せよ。

（何を変えたらどうなったかーそれはなぜかーもっとよくするにはどうしたら、どうできたらいいか？）

ヒント :
- モデル構造や学習率を変更するとき
```
# model構造を変更する場合
model = keras.Sequential(
    [
        layers.InputLayer(input_shape=3072),  # ここは変更しない
        # ここの間を変更する #
        layers.Dense(128, activation="relu"),
        # ここまで #
        layers.Dense(10, activation="softmax"),  # ここは変更しない
    ]
)
# 定義したモデルを確認
model.summary()
# 学習率の変更
lr = 0.001
model_setup(model, lr=lr)
```
- 学習のパラメータ(バッチサイズやエポック数)を変更するとき
```
epochs = 50
batch_size = 256
log = model.fit(
    X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1
)
```
- 評価するとき
```
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print("誤差　: ", loss)
print("正解率 : ", accuracy)
```
- より発展的な内容にチャレンジする場合はkerasの公式ページを確認する。
   - https://keras.io/ja/

In [ ]:
# 以下、各自の実験コードは以下のセルを活用して自由に試してみよ。（自由に増やしてよい）

ここの部分に、試したことと、その結果を以下にまとめる。

 -
 -